## Project 3
### Scrape data from CDC Opioid Deaths website

In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
# import mplcursors
import pandas as pd
# from datetime import datetime
# from datetime import timezone
# import numpy as np
import os
import glob
import time
import requests
import bs4

[CDC Drug Overdose Deaths Site](https://www.cdc.gov/drugoverdose/deaths/index.html). Inspect the page to identify which elements to scrape.

In [2]:
# https://www.cdc.gov/drugoverdose/deaths/index.html
# this is the baseurl - needs year concatenated to get to the data pages
basestateUrl = 'https://www.cdc.gov/drugoverdose/deaths/'


In [3]:
# list of years to use with the baseurl to create a working url
years = [2014, 2015, 2016, 2017, 2018, 2019, 2020]
# years = [2020]

In [4]:
# https://gist.github.com/rogerallen/1583593
#  added District Of Columbia
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    # "District of Columbia": "DC",
    "District Of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [5]:
# this opens the test browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


need to start for loop here



In [6]:
# THIS WILL BE A HUGE CHUNK 
# check url and visit website

for year in years:

    url = f'{basestateUrl}{year}.html'
    print(url)
    browser.visit(url)

    time.sleep(5)
    
    # read the loaded html into a table
    html = browser.html
    tablelist = pd.read_html(html)
    tabledf = tablelist[0]
    # https://www.geeksforgeeks.org/rename-column-by-index-in-pandas/
    tabledf.columns.values[0] = 'State'
    tabledf.columns.values[1] = 'Range_category'
    tabledf.columns.values[2] = 'Age_adjusted_rate'
    tabledf.columns.values[3] = 'Number_of_Deaths'
    tabledf['Year'] = year
    tabledf['Abbreviation'] = tabledf['State'].map(us_state_to_abbrev)
    tabledff=tabledf[['State','Abbreviation','Age_adjusted_rate','Number_of_Deaths','Year']]
    tabledff.to_csv(f'Resources/ODs/DeathState{year}.csv', index = False)


https://www.cdc.gov/drugoverdose/deaths/2014.html
https://www.cdc.gov/drugoverdose/deaths/2015.html
https://www.cdc.gov/drugoverdose/deaths/2016.html
https://www.cdc.gov/drugoverdose/deaths/2017.html
https://www.cdc.gov/drugoverdose/deaths/2018.html
https://www.cdc.gov/drugoverdose/deaths/2019.html
https://www.cdc.gov/drugoverdose/deaths/2020.html


In [7]:
# close browser
browser.quit()

In [8]:
# get list of csvs in State directory
allFiles = os.listdir("Resources/ODs")    
csvFiles = list(filter(lambda f: f.endswith('.csv'), allFiles))
csvFiles

['DeathState2014.csv',
 'DeathState2015.csv',
 'DeathState2016.csv',
 'DeathState2017.csv',
 'DeathState2018.csv',
 'DeathState2019.csv',
 'DeathState2020.csv']

In [9]:
# create list of dataframes
dfs = []
for csvFile in csvFiles:
    dfs.append(pd.read_csv(f'Resources/ODs/{csvFile}'))

In [10]:
#  check some dfs in list
print(dfs[3].head(2))
dfs[6].head(2)

     State Abbreviation  Age_adjusted_rate  Number_of_Deaths  Year
0  Alabama           AL               18.0               835  2017
1   Alaska           AK               20.2               147  2017


,State,Abbreviation,Age_adjusted_rate,Number_of_Deaths,Year
0,Alabama,AL,22.3,1029,2020
1,Alaska,AK,22.0,160,2020


In [11]:
# check return on year
yearcheck = dfs[6]['Year'].min()
yearcheck

2020

In [12]:
# dlite will be used to recreate csv for Brandon's choropleth
dlite = []
for d in dfs:
    year = d['Year'].min()
    dd = d.rename(columns = {'Number_of_Deaths':f'{year}_Number_of_Deaths','Abbreviation': 'code','State':'state'}) 
    # ddd = dd.drop(columns = ['Year','Age_adjusted_rate'])
    dd['category'] = 'state'
    ddd = dd[['code','state','category',f'{year}_Number_of_Deaths']]
    dlite.append(ddd)
    

In [13]:
len(dlite)

7

In [14]:
#  check some dfs in list
print(dlite[3].head(2))
dlite[4].head(2)

  code    state category  2017_Number_of_Deaths
0   AL  Alabama    state                    835
1   AK   Alaska    state                    147


,code,state,category,2018_Number_of_Deaths
0,AL,Alabama,state,775
1,AK,Alaska,state,110


In [15]:

wideFramedf0 = pd.merge(dlite[0],dlite[1], on=['code','state','category'], how = 'outer')
wideFramedf1 = pd.merge(dlite[2],dlite[3], on=['code','state','category'], how = 'outer')
wideFramedf2 = pd.merge(dlite[4],dlite[5], on=['code','state','category'], how = 'outer')
# wideFramedf3 = pd.merge(dlite[6],dlite[7], on=['code','state'], how = 'outer')
# wideFramedf4 = pd.merge(dlite[8],dlite[9], on=['code','state'], how = 'outer')
# wideFramedf5 = pd.merge(dlite[10],dlite[11], on=['code','state'], how = 'outer')
# wideFramedf6 = pd.merge(dlite[12],dlite[13], on=['code','state'], how = 'outer')


In [16]:
wf0 = pd.merge(wideFramedf0,wideFramedf1, on=['code','state','category'], how = 'outer')
wf1 = pd.merge(wideFramedf2,dlite[6], on=['code','state','category'], how = 'outer')
# wf2 = pd.merge(wideFramedf4,wideFramedf5, on=['code','state'], how = 'outer')

In [17]:
# this merge level not needed
# wff0 = pd.merge(wf0,wf1, on=['code','state'], how = 'outer')
# wff1 = pd.merge(wf2, wideFramedf6, on=['code','state'], how = 'outer')

In [18]:
wideframe = pd.merge(wf0, wf1, on=['code','state','category'], how = 'outer')
wideframe['total_deaths'] = wideframe['2014_Number_of_Deaths'] + wideframe['2014_Number_of_Deaths'] + wideframe['2015_Number_of_Deaths'] + wideframe['2016_Number_of_Deaths'] + wideframe['2017_Number_of_Deaths'] + wideframe['2018_Number_of_Deaths'] + wideframe['2019_Number_of_Deaths'] 
wideframe2 = wideframe[['code', 'state', 'category', 'total_deaths', '2014_Number_of_Deaths',
       '2015_Number_of_Deaths', '2016_Number_of_Deaths',
       '2017_Number_of_Deaths', '2018_Number_of_Deaths',
       '2019_Number_of_Deaths', '2020_Number_of_Deaths']]

In [19]:
wideframe.columns

Index(['code', 'state', 'category', '2014_Number_of_Deaths',
       '2015_Number_of_Deaths', '2016_Number_of_Deaths',
       '2017_Number_of_Deaths', '2018_Number_of_Deaths',
       '2019_Number_of_Deaths', '2020_Number_of_Deaths', 'total_deaths'],
      dtype='object')

In [27]:

# this creates OpioidDeathsAll-cleaned-3.csv  and this data will be transposed for use in the barchart
wideframe2.to_csv('Resources/ODs/OpioidDeathsWideframe.csv', index = False)
wideframe2

,code,state,category,total_deaths,2014_Number_of_Deaths,2015_Number_of_Deaths,2016_Number_of_Deaths,2017_Number_of_Deaths,2018_Number_of_Deaths,2019_Number_of_Deaths,2020_Number_of_Deaths
0,AL,Alabama,state,5316,723,736,756,835,775,768,1029
1,AK,Alaska,state,887,124,122,128,147,110,132,160
2,AZ,Arizona,state,10187,1211,1274,1382,1532,1670,1907,2550
3,AR,Arkansas,state,2783,356,392,401,446,444,388,546
4,CA,California,state,34769,4521,4659,4654,4868,5348,6198,8908
5,CO,Colorado,state,6698,899,869,942,1015,995,1079,1492
6,CT,Connecticut,state,6372,623,800,971,1072,1069,1214,1371
7,DE,Delaware,state,2032,189,198,282,338,401,435,444
8,DC,District Of Columbia,state,1461,96,125,269,310,254,311,424
9,FL,Florida,state,28278,2634,3228,4728,5088,4698,5268,7231


In [22]:
# update table to prep for transposing in next step
wf = wideframe.drop(columns = ['state','category','total_deaths'])
wff = wf.rename(columns = {'code':'Overdose_Deaths_per_Year','2014_Number_of_Deaths':'2014',
       '2015_Number_of_Deaths':'2015', '2016_Number_of_Deaths':'2016',
       '2017_Number_of_Deaths':'2017', '2018_Number_of_Deaths':'2018',
       '2019_Number_of_Deaths':'2019', '2020_Number_of_Deaths':'2020'})


wfff = wff.set_index('Overdose_Deaths_per_Year')# to prepare for next step - transpose table

In [23]:
wfff.head(2)

,2014,2015,2016,2017,2018,2019,2020
Overdose_Deaths_per_Year,,,,,,,
AL,723,736,756,835,775,768,1029
AK,124,122,128,147,110,132,160


In [24]:
# transpose table into format for barchart use and save to csv
wideframetrans = pd.DataFrame(wfff).transpose().reset_index()
# wideframetrans
# wft = wideframetrans.reset_index()
wft= wideframetrans.rename(columns = {'index': 'Year'})
wft.to_csv('Resources/ODs/OpioidDeathsAllTransposed.csv', index = False)
wft.head(2)

Overdose_Deaths_per_Year,Year,AL,AK,AZ,AR,CA,CO,CT,DE,DC,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,2014,723,124,1211,356,4521,899,623,189,96,...,63,1269,2601,603,83,980,979,627,853,109
1,2015,736,122,1274,392,4659,869,800,198,125,...,65,1457,2588,646,99,1039,1094,725,878,96


In [25]:
# create json file from csv - json will be used for interactive barcharts on the website
df = pd.read_csv (r'Resources/ODs/OpioidDeathsAllTransposed.csv')
df.to_json (r'Resources/ODs/OpioidDeathsAllTransposed.json')

In [26]:
# do we need this?

bigFrame = pd.concat(dfs, ignore_index = True)
bigFrame.to_csv('Resources/ODs/OpioidDeathsbigFrame.csv', index = False)
# bigFrame = bigFrame2[['State', 'Abbreviation', 'Opioid Dispensing Rate per 100', 'Year']]
print(bigFrame)


             State Abbreviation  Age_adjusted_rate  Number_of_Deaths  Year
0          Alabama           AL               15.2               723  2014
1           Alaska           AK               16.8               124  2014
2          Arizona           AZ               18.2              1211  2014
3         Arkansas           AR               12.6               356  2014
4       California           CA               11.1              4521  2014
..             ...          ...                ...               ...   ...
352       Virginia           VA               26.6              2240  2020
353     Washington           WA               22.0              1733  2020
354  West Virginia           WV               81.4              1330  2020
355      Wisconsin           WI               27.7              1531  2020
356        Wyoming           WY               17.4                99  2020

[357 rows x 5 columns]
